In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys, pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

sys.path.append('/Users/dp/pma/')

from importlib import reload
import sameRiver
from sameRiver import *

import sameRiver.countsO
import sameRiver.scheme
importlib.reload(sameRiver.countsO)


In [ ]:
def multiply_by_xl_rate(df):
    xl_rate = '/Users/dp/pma/dataAndScripts/tables/percentCrosslinked.xlsx'

    xl = pandas.read_excel(xl_rate)
    xl = xl[xl['Label']=='% XL (monomeric)'].copy()
    m = xl.groupby('Protein').mean()
    print(m)
    to_xl = dict(zip(m.index, m.Value))

    #to_xl['No vector'] = 0.002
    to_xl['hnRNPC'] = to_xl['hnRNP C']

    _df = df.copy()

    by_rate = _df.copy()
    for col in _df.columns:

        if col == 'gene_name' or col == 'Gene type':
            continue
        gene = col.split('_')[-2]

        if gene in to_xl:
            by_rate[col] = 0.01 * to_xl[gene] * _df[col]
        else:
            del by_rate[col]

    #print(by_rate.iloc[:2, :2])

    return by_rate

def per_million(df):
    _df = df.copy()
    for col in df.columns:
        if col in ['gene_name', 'Gene name', 'gene', 'Gene type']:
            continue
       # print(col, _df[col].sum())
        try:
            _df[col] = 1E6 * _df[col]/_df[col].sum()
        except:
            print("Falied to convert to per mill: ", col)
        
    return _df

#per_million(df)
#arrays = make_gauss_figure(by_rate)

In [ ]:
def smooth_y_values(arr):

    pos_hits = []
    print('max value {}'.format(np.max(arr)))
    pos = [_ for _ in arr if _>0]
    min_pos = np.min(pos)
    
    if len(pos) == 0:
        print("No counts passed. Empty array given to smooth_y_values.")
        return 
    
    if min_pos >= 1:
        ratio_factor = 1
    else:
        ratio_factor = int(1/min_pos)
        
    print('ratio_factor = {}'.format(ratio_factor))
    
#    ratio_factor = np.min([ratio_factor, 1E])
    
    for pos, val in enumerate(arr):
        if val > 0:
            pos_hits.extend([pos] * int(ratio_factor*val))
            

    if len(pos_hits) > 5000:
        print('randoming: {}'.format(arr[:20]))
        pos_hits = random.sample(pos_hits, 5000)

    if len(arr) < 200:
        bw = 5
    else:
        bw = 20

    kde = KernelDensity(
        kernel='gaussian', bandwidth=bw).fit(np.array(pos_hits)[:, np.newaxis])

    x = np.linspace(0, len(arr)-1, len(arr))
    y = kde.score_samples(x[:, np.newaxis])
    y = [10**_ for _ in y]
    print("Max value in arr: {}. Max value in y: {}. Ratio: {}".format(
        np.max(arr), np.max(y), np.max(arr)/np.max(y)))
    ratio = np.max(arr)/np.max(y)
    print(ratio)
    y = [ratio * _ for _ in y]
    return x, y

In [ ]:
######### EXAMPLE. NOT CLIP DATA.


N = 100
np.random.seed(1)
X = np.concatenate(
                    (
                        np.random.normal(0, 1, int(0.3 * N)),
                        #np.random.normal(5, 1, int(0.7 * N))
                    )
                  )[:, np.newaxis]

#print(X)
X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]

#true_dens = (0.3 * norm(0, 1).pdf(X_plot[:, 0])
#             + 0.7 * norm(5, 1).pdf(X_plot[:, 0])
#            )
#print(true_dens)
fig, ax = plt.subplots()
#ax.fill(X_plot[:, 0], true_dens, fc='black', alpha=0.2,
#        label='input distribution')

for kernel in ['gaussian', 'tophat', 'epanechnikov']:
    kde = KernelDensity(kernel=kernel, bandwidth=0.5).fit(X)
    log_dens = kde.score_samples(X_plot)
    ax.plot(X_plot[:, 0], np.exp(log_dens), '-',
            label="kernel = '{0}'".format(kernel))

ax.text(6, 0.38, "N={0} points".format(N))

ax.legend(loc='upper left')
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k')

ax.set_xlim(-4, 9)
ax.set_ylim(-0.02, 0.4)
plt.show()

################ END EXAMPLE

In [ ]:

importlib.reload(sameRiver.countsO)

def make_gauss_figure(df):

    fig, ax = plt.subplots()

    cols = df.columns
    added_labels = set()

    arrays_to_plot = []
    genes_plotted = set()
    
    print('cols to plot: ', cols)
    for col in cols:

        if col == 'gene_name' or col == 'Gene type':
            continue
            
        print(col)

        gene = col.split('_')[-2]

        linestyle = '-'

        if gene in ['ITPA', 'CDK4',
                    'UBA2', 
                    'ETS2', 'DCNT6', 'IDE', 'VPS24', 'CAPNS6', 'TPGS2', 
                    'EPB41L5', 
                    'CCIN',
                    ]:
            color = 'k'
            alpha = 0.2
            label = 'Random protein'
        elif gene in ['FBL', 'hnRNPC']:#, 'PCBP1 L100P', 'PCBP1 L100Q', 'PCBP1']:
            color = 'r'
            alpha = 0.4
            label = 'RBP'
        elif gene in ['No vector']:
            color = 'b'
            alpha = 0.2
            label = 'No vector'
        else:
            color = 'g'
            alpha = 0.1
            label = 'Other'
            continue
            
        if label in added_labels:
            label = None
        else:
            added_labels.add(label)

        pre_clipping = np.array(
            [x for x in np.nan_to_num(df[col])]# if (
                #x>-100 and 
        #        np.isfinite(x))]
        )
        
        if gene in genes_plotted:
            continue
        print(gene)

        genes_plotted.add(gene)
#        print(df[col][df[col]>1E4])

        raw_reads = np.clip(pre_clipping, 0, 5)
        
        #logged = np.log2(logged)
        #logged = logged[np.isfinite(logged)]

        print(raw_reads[:, np.newaxis][:2])
        
        try:
            kde = KernelDensity(
                kernel='gaussian',
                bandwidth=1).fit(raw_reads[:, np.newaxis])
        except:
            print("Failed in call to KernelDensity")
        
        xaxis = np.linspace(0, 5, 200)[:, np.newaxis]

#        print(logged.shape)
#        print(xaxis.shape)
        log_dens = kde.score_samples(xaxis)
        y = [10**_ for _ in log_dens]
        
        auc = 0
        width = (xaxis[-1]-xaxis[0])/len(xaxis)
        for val in y:
            auc += val
        auc = auc * width
        print("AUC = ", auc)
            
        ratio = len(raw_reads)/auc
        y = [ratio * _ for _ in y]

        arrays_to_plot.append({
            'x': xaxis[:, 0],
            'y': y,
            'pre_clipped_rpg': pre_clipping,
            'c': color,
            'alpha': alpha,
            'label': label,
            'gene': gene,
        })
        
        ax.plot(xaxis[:, 0], y, linestyle,
                #c=color, 
                alpha=alpha, label=gene,
               )
               #label="kernel = '{0}'".format(kernel))
        #    plt.hist(logged, c='k')#, bins=100)
        #if len(added_labels) == 3:
        #    break
        
    plt.title('')
    plt.xlabel('Frequency of XL (per million proteins)')
    plt.ylabel('Instances')
    plt.legend()
    #plt.xlim(0, 200)
    plt.savefig('/Users/dfporter/pma/dataAndScripts/clip/figs/kde_of_hist_of_binding_per_RNA.svg')
    plt.show()

    return arrays_to_plot
    #'''

df = pandas.read_csv('/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/ann_counts.txt', sep='\t',
                    index_col=0)
df['gene_name'] = df.index

per_mill = per_million(df)
xl = multiply_by_xl_rate(per_mill)
#to_biotype = dict(zip(df['gene_name'], df['Gene type']))
#df['Gene type'] = [to_biotype.get(_, '') for _ in df.index]

def plot_em(df):
    mRNA = df[df['Gene type']=='protein_coding'].copy()
    mRNA_arrays_to_plot = make_gauss_figure(mRNA)

    snoRNA = df[df['Gene type']=='snoRNA'].copy()
    snoRNA_arrays_to_plot = make_gauss_figure(snoRNA)

 #   ncRNA = df[df['Gene type']!='protein_coding'].copy()
  #  ncRNA_arrays_to_plot = make_gauss_figure(ncRNA)
    
    
plot_em(xl)



In [ ]:
import random

def plot_and_get_modified(arrays_to_plot):
    fig, ax = plt.subplots()


    alt_arrays_to_plot = []
    for d in arrays_to_plot:
        _rpg = np.clip(d['pre_clipped_rpg'], 0, 1E3)

        rpg = _rpg[:, np.newaxis]

        kde = KernelDensity(kernel='gaussian',
                            bandwidth=20
                               ).fit(rpg)


        xaxis = np.linspace(0, 5E3, 1000)[:, np.newaxis]

        log_dens = kde.score_samples(xaxis)
        y = [10**_ for _ in log_dens]
        ratio = np.max(_rpg)/np.max(y)
        y = [ratio * _ for _ in y]

        # Save this version for future plotting.
        _d = dict(zip(d.keys(), d.values()))
        _d.update({
            'x': xaxis, 'y': y
        })
        alt_arrays_to_plot.append(_d)

        ax.plot(xaxis, np.exp(log_dens), 'k-',
                    c=d['c'], alpha=d['alpha'], label=d['gene'])

        if random.randint(0, 20) == 1:
            print('plotted {} for {} RNAs'.format(d['gene'], rpg.shape))

    #plt.ylim(0, 0.0005)
    #plt.xlim(0, 1000)
    plt.title('')
    plt.xlabel('Frequency of XL (per million, log10)')
    plt.ylabel('Instances')
    plt.legend()
    #plt.savefig('/Users/dfporter/pma/dataAndScripts/clip/figs/kde_of_hist_of_binding_per_RNA.svg')
    plt.show()
    plt.clf()
    plt.close()
    return alt_arrays_to_plot

alt_mRNA_arrays_to_plot = plot_and_get_modified(mRNA_arrays_to_plot)
alt_snoRNA_arrays_to_plot = plot_and_get_modified(snoRNA_arrays_to_plot)
alt_ncRNA_arrays_to_plot = plot_and_get_modified(ncRNA_arrays_to_plot)

In [ ]:
def re_plot(_arrays_to_plot, figlabel):
    plt.clf()
    fig, ax = plt.subplots()

    genes_plotted = set()
    for d in _arrays_to_plot:
        
        if d['gene'] in genes_plotted:
            continue
            
        if d['gene'] in to_xl:
            x = 0.01 * to_xl[d['gene']] * d['x']
            y = d['y']
        else:
            continue
            
        ax.plot(x, y, '-',
                    #c=d['c'],
                alpha=d['alpha'], label=d['gene'])

        genes_plotted.add(d['gene'])
        
    #plt.ylim(0, 100)
    plt.xlim(0, 30)
    plt.title('')
    plt.xlabel('Crosslinks at an RNA per million proteins')
    plt.ylabel('Instances (KDE)')
    plt.legend()
    plt.savefig(
        '/Users/dfporter/pma/dataAndScripts/clip/figs/kde_of_hist_of_{}_binding_per_RNA.pdf'.format(
            figlabel))
    plt.show()
    plt.clf()
    plt.close()
    
re_plot(alt_mRNA_arrays_to_plot, 'mRNA')
re_plot(alt_snoRNA_arrays_to_plot, 'snoRNA')
re_plot(alt_ncRNA_arrays_to_plot, 'ncRNA')